In [1]:
from tkinter import *
import subprocess
import psutil
import json
from tkinter.ttk import Combobox 
from tkinter import simpledialog 
from PIL import Image, ImageTk
from tkinter.scrolledtext import ScrolledText


sudo_password = None
report_root = None

def run_command(command):
    global sudo_password

    if sudo_password is None:
        sudo_password = simpledialog.askstring("Password", "Enter your sudo password:", show='*')

    if sudo_password:
        full_command = f"echo {sudo_password} | sudo -S {command}"
        result = subprocess.run(full_command, shell=True, text=True, capture_output=True)
        if result.returncode != 0:  # Check if the command failed
            sudo_password = None  # Reset the password
            if "command not found" in result.stderr:  # Check if command not found error occurred
                return "Error: Command not recognized."
            else:
                return "Error: Command failed."
        else:
            return result.stdout 
    else:
        return "Password not provided."
    
def display_output(output):
    output_text.delete(1.0, END)  # Clear the previous output
    output_text.insert(END, output)
############################################################################# main button commands
def llist():
    devices = run_command("lspci")
    list_box.delete(0, END)
    
    # Define a list of background colors for the rainbow effect
    rainbow_colors = ["#EEEEEE", "#DDDDDD", "#CCCCCC", "#BBBBBB", "#AAAAAA"]
    
    for i, device in enumerate(devices.splitlines()):
        # Set the background color for each item using the rainbow_colors list
        bg_color = rainbow_colors[i % len(rainbow_colors)]
        list_box.insert(END, device)
        list_box.itemconfig(i, {'bg': bg_color})
####################################################################################       

def run_lspci(option):
    selected_device = list_box.get(ACTIVE)
    if selected_device:
        command = f"lspci {option} -s {selected_device.split()[0]}"
        output = run_command(command)
        display_output(output)

# Primary button commands
def lspci_s():
    run_lspci("")

def lspci_x():
    run_lspci("-x")

def lspci_k():
    run_lspci("-k")

def lspci_v():
    run_lspci("-v")

def lspci_n():
    run_lspci("-n")

def lspci_vmm():
    run_lspci("-vmm")

def lspci_vvv():
    run_lspci("-vvv")    

###############################################################
def clear_output():
    output_text.delete(1.0, END)  # Clear the output text
########################################################################################    
def run_search_command():
    command = search_entry.get().strip()   # Get the command from the search entry
    output = run_command(command)  #Execute the command and capture its output
    output_text.delete(1.0, END)  # Clear existing text
    output_text.insert(END, output)  # Display the command output in the text box
########################################################################################
def collect_data():
    cpu_percent = psutil.cpu_percent()
    mem_percent = psutil.virtual_memory().percent
    return cpu_percent, mem_percent

def update_data():
    cpu_percent, mem_percent = collect_data()
    cpu_label.config(text=f"CPU Usage: {cpu_percent}%", fg="white", bg="#8B3A62")  # Set the text color to red and background color to the desired pinkish color
    memory_label.config(text=f"Memory Usage: {mem_percent}%", fg="white", bg="#8B3A62")  # Set the text color to black and background color to the desired pinkish color
    root.after(800, update_data)

# Create the main Tkinter window
root = Tk()
root.title("Linux Terminal Utility")
root.geometry("1920x1080")
root.configure(bg="#545454")
######################################################################################

# Load your image using PIL (replace with the actual path to your image)
image = Image.open("amd-logos-web-update-2020_4_678x452.png")

# Define the desired width and height for the resized image
new_width = 250  # Adjust this to your preferred width
new_height = 140  # Adjust this to your preferred height

# Resize the image to the desired dimensions
image = image.resize((new_width, new_height), Image.ANTIALIAS)

# Convert the resized image to a format that tkinter can display
tk_image = ImageTk.PhotoImage(image)  # Use ImageTk.PhotoImage

image_label = Label(root, image=tk_image)
image_label.image = tk_image  # Make sure to keep a reference to the image
image_label.place(x=1095, y=126)
###########################################################################################################
search_entry = Entry(root, width=30, )
search_entry.pack(side=TOP, padx=460, pady=10, anchor="nw")  # anchor="nw" sets the anchor to top-left

# Set the background color of the search button to green
run_button = Button(root, text="Search", command=run_search_command)
run_button.pack(side=TOP, padx=550, pady=0, anchor="nw") 

# ###############################################Create and place the header label
header_label = Label(root, text="PCI Express & Devices", font=("Times New Roman", 20, "bold"), bg="#545454", fg="white")
header_label.pack(side=TOP, padx=450, pady=0, anchor="nw")
####################################################################################create main interpret button

def open_report_gui():
    global report_root, output_txt, sudo_password
    
    report_root = Tk()
    report_root.title("PCI Express Configuration")
    report_root.geometry("500x480")  # Adjust the dimensions as needed
    report_root.configure(bg="black")  # Set GUI background color to black
    
    canvas = Canvas(report_root, bg="black")  # Create a canvas for window content
    canvas.pack(fill="both", expand=True)

    def display_output(output):
        text_box.delete(1.0, "end")  # Clear the previous output
        text_box.insert("end", output)
        
    def display_out(output):
        output_txt.delete(1.0, "end")  # Clear the previous output
        output_txt.insert("end", output)

    def parse_lspci_output(output):
        lines = output.split('\n')
        if len(lines) < 2:
            return None, None, None, None, None, None, None, None, None, None

        hexdump_line = lines[1]
        hexdump_values = hexdump_line.split()[1:]

        # Swap the position of the digits for each pair of hex values
        swapped_values = [value[2:] + value[:2] for value in hexdump_values]
        swapped_values1 = [value[2:] + value[:2] for value in hexdump_values]
        swapped_values2 = [value[2:] + value[:2] for value in hexdump_values]
        swapped_values3 = [value[2:] + value[:2] for value in hexdump_values]
        
        
        swapped_values = swapped_values[1:2] + swapped_values[0:1] + swapped_values[3:4] + swapped_values[2:3]
        swapped_values1 = swapped_values1[5:6] + swapped_values1[4:5] + swapped_values1[7:8] + swapped_values1[6:7]
        swapped_values2 = swapped_values2[11:12] + swapped_values2[9:10] + swapped_values2[8:9] + swapped_values2[10:11]
        swapped_values3 = swapped_values3[14:15] + swapped_values3[13:14] + swapped_values3[12:13] + swapped_values3[15:16]

        # Extract values from the second line
        vendor_id = ''.join(swapped_values[:2])           
        device_id = ''.join(swapped_values[2:4])
        Control = ''.join(swapped_values1[:2])  
        status = ''.join(swapped_values1[2:4])  
        Class_Code = ''.join(swapped_values2[1:7])  # Extract first 6 values for Class Code
        Revision_ID = ''.join(swapped_values2[:1])  # Extract next 2 values for Revision ID

        BIST = ''.join(swapped_values3[:1])  ## extract the next 2 values 
        Header_Type = ''.join(swapped_values3[2:3])  ## extract the next 2 values 
        Latency_Timer = ''.join(swapped_values3[1:2])  ## extract the next 2 values 
        Cache_Line_Size = ''.join(swapped_values3[3:4])  # Extract the next 2 values

        return device_id, vendor_id, status, Control, Class_Code, Revision_ID, BIST, Header_Type, Latency_Timer, Cache_Line_Size
    
    
    def parse_lspci_output2(output):
        lines = output.split('\n')
        if len(lines) < 3:  # Ensure there are at least 3 lines in the output
            return None, None, None, None

        hexdump_line_3 = lines[2]  # Third line containing hex values
        hexdump_values_3 = hexdump_line_3.split()[1:]  # Extract hex values from the third line

        # Rearrange the position of the digits for each pair of hex values in the base address
        base_address_rearranged = hexdump_values_3[3] + hexdump_values_3[2] + hexdump_values_3[1] + hexdump_values_3[0]
        # Extract other values
        Cardbus_Cis = ''.join([value[2:] + value[:2] for value in hexdump_values_3[4:6]])
        Subsystem_ID = ''.join([value[2:] + value[:2] for value in hexdump_values_3[6:8]])
        Subsystem_Vendor = ''.join([value[2:] + value[:2] for value in hexdump_values_3[10:12]])
        return base_address_rearranged, Cardbus_Cis, Subsystem_ID, Subsystem_Vendor

    
    def display_device_info(event=None):
        selected_device = device_list.get()
        result = subprocess.run(['lspci', '-x'], capture_output=True, text=True)
        output = result.stdout

        # Find the section corresponding to the selected device
        start_index = output.find(selected_device)
        if start_index != -1:
            section = output[start_index:]
            end_index = section.find('\n\n')
            if end_index != -1:
                section = section[:end_index]
                device_info = parse_lspci_output(section)
                device_info2 = parse_lspci_output2(section)
                if device_info and device_info2:
                    device_id, vendor_id, status, Control, Class_Code, Revision_ID, BIST, Header_Type, Latency_Timer, Cache_Line_Size = device_info
                    Base_address, Cardbus_Cis, Subsystem_ID, Subsystem_Vendor = device_info2
                    # Update the labels after a delay of 1 second
                    root.after(1000, update_labels, device_id, vendor_id, status, Control, Class_Code, Revision_ID, BIST, Header_Type, Latency_Timer, Cache_Line_Size, Base_address, Cardbus_Cis, Subsystem_ID, Subsystem_Vendor)

                       
    def update_labels(device_id, vendor_id, status, Control, Class_Code, Revision_ID, BIST, Header_Type, Latency_Timer, Cache_Line_Size, Base_address, Cardbus_Cis, Subsystem_ID, Subsystem_Vendor):
        device_id_button.config(text=f"Device ID: \n{device_id}")
        vendor_id_button.config(text=f"Vendor ID: \n{vendor_id}")
        status_button.config(text=f"Status: \n{status}")
        Control_button.config(text=f"Control: \n{Control}")
        Class_Code_button.config(text=f"Class Code: \n{Class_Code}")
        Revision_ID_button.config(text=f"Revision ID: \n{Revision_ID}")
        BIST_button.config(text=f"BIST: \n{BIST}")
        Header_Type_button.config(text=f"Header Type: \n{Header_Type}")
        Latency_Timer_button.config(text=f"Master Latency Timer: \n{Latency_Timer}")
        Cache_Line_Size_button.config(text=f"Cache Line Size: \n{Cache_Line_Size}")
        Base_address_button.config(text=f"Base address: \n{Base_address}")
        Cardbus_Cis_button.config(text=f"Cardbus Cis: \n{Cardbus_Cis}")
        Subsystem_ID_button.config(text=f"Subsystem ID: \n{Subsystem_ID}")
        Subsystem_Vendor_button.config(text=f"Subsystem Vendor : \n{Subsystem_Vendor}")

        
        # Define all the binary variables
        device_id_binary = None
        vendor_id_binary = None
        status_binary = None
        Control_binary = None
        Class_Code_binary = None
        Revision_ID_binary = None
        BIST_binary = None
        Header_Type_binary = None
        Latency_Timer_binary = None
        Cache_Line_Size_binary = None
        Base_address_binary = None
        Cardbus_Cis_binary = None
        Subsystem_ID_binary = None
        Subsystem_Vendor_binary = None
        

        def pad_binary(binary_str):
            return '0' * (32 - len(binary_str)) + binary_str
        
        
        def Base_Add_clicked():
            if Base_address is not None:
                register_clicked("Base Address", "Base Address Register")
                binary_value = pad_binary(bin(int(Base_address, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("Base address is not defined.")

        def Revision_ID_clicked():
            if Revision_ID is not None:
                register_clicked("Revision ID", "Revision ID Register")
                binary_value = pad_binary(bin(int(Revision_ID, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
        
        def CIS_Pointer_clicked():
            if Cardbus_Cis is not None:
                register_clicked("CIS Pointer", "CIS pointer Register")
                binary_value = pad_binary(bin(int(Cardbus_Cis, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
        
        def Subsystem_ID_clicked():
            if Subsystem_ID is not None:
                register_clicked("Subsystem ID", "Subsystem ID Register")
                # Calculate binary value and display it
                binary_value = pad_binary(bin(int(Subsystem_ID, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
                
        def Subsystem_Vendor_clicked():
            if Subsystem_Vendor is not None:
                register_clicked("Subsystem Vendor ID", "Subsystem Vendor Register")
                binary_value = pad_binary(bin(int(Subsystem_Vendor, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
                
    ###########################################################################################            
        def BIST_clicked():
            if BIST is not None:
                register_clicked("BIST", "BIST Register")
                binary_value = pad_binary(bin(int(BIST, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
        
        def Header_Type_clicked():
            if Header_Type is not None:
                register_clicked("Header Type", "Header Type Register")
                binary_value = pad_binary(bin(int(Header_Type, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
        
        def Latency_Timer_clicked():
            if Latency_Timer is not None:
                register_clicked("Latency Timer", "Latency Timer Register")
                binary_value = pad_binary(bin(int(Latency_Timer, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
                
        def Cache_Line_clicked():
            if Cache_Line_Size is not None:
                register_clicked("Cache Line Size", "Cache Line Size Register")
                binary_value = pad_binary(bin(int(Cache_Line_Size, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
                
        def Class_Code_clicked():
            if Class_Code is not None:
                register_clicked("Class Code", "Class Code Register")
                binary_value = pad_binary(bin(int(Class_Code, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
                
##################################################################################################                
        def device_id_clicked():
            if device_id is not None:
                register_clicked("Device ID", "Device ID Register")
                binary_value = pad_binary(bin(int(device_id, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")


        def vendor_id_clicked():
            if vendor_id is not None:
                register_clicked("Vendor ID", "Vendor ID Register")
                binary_value = pad_binary(bin(int(vendor_id, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("not defined.")
##############################################################################################   

        def status_clicked():
            if status is not None:
                register_clicked("Status Entire", "Status Entire Register")
                binary_value = pad_binary(bin(int(status, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("status is not defined.")
                
        def control_clicked():
            if Control is not None:
                register_clicked ("Control Entire", "Control Full Register")
                binary_value = pad_binary(bin(int(Control, 16))[2:])
                update_binary_display(binary_value)
            else:
                print("Control is not defined.")

        device_id_button.config(command=device_id_clicked)
        vendor_id_button.config(command=vendor_id_clicked)
        status_button.config(command=status_clicked)
        Control_button.config(command=control_clicked)
        Class_Code_button.config(command=Class_Code_clicked)
        Revision_ID_button.config(command=Revision_ID_clicked)
        BIST_button.config(command=BIST_clicked)
        Header_Type_button.config(command=Header_Type_clicked)
        Latency_Timer_button.config(command=Latency_Timer_clicked)
        Cache_Line_Size_button.config(command=Cache_Line_clicked)
        Base_address_button.config(command=Base_Add_clicked)
        Cardbus_Cis_button.config(command=CIS_Pointer_clicked)
        Subsystem_ID_button.config(command=Subsystem_ID_clicked)
        Subsystem_Vendor_button.config(command=Subsystem_Vendor_clicked)
        
    #############################################################################################################
    def print_json_info(json_data, keys):
        info = json_data[keys[0]][keys[1]]
        output = ""
        for i, line in enumerate(info):
            if i == 0:
                output += f">> {line}\n"
            else:
                output += f"     {line}\n"
        output += "\n"  # Add two empty lines between each set of data
        return output
    ############################################################################################################

    def status_clicked11():
        global output_txt  # Declare output_txt as global
        with open("PCIe Register Data.json", "r") as file:
            jsonData = json.load(file)
        output = ""
        output += print_json_info(jsonData, ["Status", "Status Register"])
        display_out(output)

    def control_clicked11():
        global output_txt  # Declare output_txt as global
        with open("PCIe Register Data.json", "r") as file:
            jsonData = json.load(file)
        output = ""
        output += print_json_info(jsonData, ["Control", "Control Register"])
        display_out(output)


###############################################################################################################         
    def register_clicked(register_name, register_display_name):
        with open("PCIe Register Data.json", "r") as file:
            jsonData = json.load(file)

        output = print_json_info(jsonData, [register_name, register_display_name])
        display_output(output)
##################################################################################################        
    def status_clicked():
        register_clicked("Status Entire", "Status Entire Register")
    def control_clicked():
        register_clicked ("Control Entire", "Control Full Register")
######################################################################################################
    def device_id_clicked():
        register_clicked("Device ID", "Device ID Register")

    def vendor_id_clicked():
        register_clicked("Vendor ID", "Vendor ID Register")

    def Base_Add_clicked():
        register_clicked("Base Address", "Base Address Register")

    def Cache_Line_clicked():
        register_clicked("Cache Line Size", "Cache Line Size Register")

    def Header_Type_clicked():
        register_clicked("Header Type", "Header Type Register")

    def Latency_Timer_clicked():
        register_clicked("Latency Timer", "Latency Timer Register")

    def Class_Code_clicked():
        register_clicked("Class Code", "Class Code Register")

    def BIST_clicked():
        register_clicked("BIST", "BIST Register")

    def Revision_ID_clicked():
        register_clicked("Revision ID", "Revision ID Register")

    def CIS_Pointer_clicked():
        register_clicked("CIS Pointer", "CIS pointer Register")

    def Subsystem_ID_clicked():
        register_clicked("Subsystem ID", "Subsystem ID Register")

    def Subsystem_Vendor_clicked():
        register_clicked("Subsystem Vendor ID", "Subsystem Vendor Register")

##############################################################################################################
    device_id_button = Button(report_root, text="Device ID: ", command=device_id_clicked, height=2,
                              width=15, fg="red", bg="black")
    device_id_button.place(x=80, y=20)

    vendor_id_button = Button(report_root, text="Vendor ID: ", command=vendor_id_clicked, height=2,
                              width=15, fg="red", bg="black")
    vendor_id_button.place(x=230, y=20)

    status_button = Button(report_root, text="Status: ", command=status_clicked, height=2,
                           width=15, fg="red", bg="black")
    status_button.place(x=80, y=70)

    Control_button = Button(report_root, text="Control: ", command=control_clicked, height=2,
                            width=15, fg="red", bg="black")
    Control_button.place(x=230, y=70)
############################################################  
    status_Location = Button(report_root, text="[#]", command=status_clicked11, height=2, 
                             width=1, fg="red", bg="black")
    status_Location.place(x=44, y=70)

    control_Location = Button(report_root, text="[#]",command=control_clicked11, height=2, 
                              width=1, fg="red", bg="black")
    control_Location.place(x=378, y=70)
##################################################################
    Class_Code_button = Button(report_root, text="Class Code: ", command=Class_Code_clicked, height=2,
                           width=25, fg="red", bg="black")
    Class_Code_button.place(x=60, y=120)

    Revision_ID_button= Button(report_root, text="Revision ID: ", command=Revision_ID_clicked, height=2,
                            width=11, fg="red", bg="black")
    Revision_ID_button.place(x=290, y=120)
    
    BIST_button= Button(report_root, text="BIST: ", command=BIST_clicked, height=2,
                            width=15, fg="red", bg="black")
    BIST_button.place(x=80, y=170)
    
    Header_Type_button= Button(report_root, text="Header type: ", command=Header_Type_clicked, height=2,
                            width=15, fg="red", bg="black")
    Header_Type_button.place(x=230, y=170)
    
    Latency_Timer_button= Button(report_root, text="Master Latency timer:", command=Latency_Timer_clicked, height=2,
                            width=17, fg="red", bg="black")
    Latency_Timer_button.place(x=70, y=220)
    
    Cache_Line_Size_button= Button(report_root, text="Cache Line Size:", command=Cache_Line_clicked, height=2,
                            width=16, fg="red", bg="black")
    Cache_Line_Size_button.place(x=235, y=220)
    
    Base_address_button= Button(report_root, text= "Base Address Registers:", command=Base_Add_clicked, height=3,
                            width=30, fg="red", bg="black")
    Base_address_button.place(x=100, y=270)
    
    Cardbus_Cis_button= Button(report_root, text="Cardbus CIS Pointer:", command=CIS_Pointer_clicked, height=2,
                            width=30, fg="red", bg="black")
    Cardbus_Cis_button.place(x=100, y=337)
    
    Subsystem_ID_button= Button(report_root, text="Subsystem ID:", command=Subsystem_ID_clicked, height=2,
                            width=16, fg="red", bg="black")
    Subsystem_ID_button.place(x=76, y=387)
    
    Subsystem_Vendor_button= Button(report_root, text="Subsystem Vendor ID:", command=Subsystem_Vendor_clicked, height=2,
                            width=16, fg="red", bg="black")
    Subsystem_Vendor_button.place(x=234, y=387)
    
    
    # Get the lspci devices
    result = subprocess.run(['lspci'], capture_output=True, text=True)
    output = result.stdout
    devices = [line.strip() for line in output.split('\n') if line.strip()]

    device_list = Combobox(report_root, values=devices, state="readonly", width=30)
    device_list.pack()

    # Bind the combobox selection event to display_device_info function
    device_list.bind("<<ComboboxSelected>>", display_device_info)
    
    # Create a frame to hold the labels
    label_frame = Frame(report_root, bg="black")
    label_frame.place(x=430, y=18)  # Adjust the position as needed

    # List of labels text
    label_texts = ["00h", "04h", "08h", "0Ch", "0Ch" , "10h-24h" , "28h" , "2Ch"]

    # Create labels and add them to the frame
    for text in label_texts:
        label = Label(label_frame, text=text, bg="black", fg="white")
        label.pack(anchor="e", pady=16)  # Align labels to the right
    report_root.mainloop()
                                 
# Create the report button in the root window with specified color and text
report_button = Button(report_root, text="Report Interpret", command=open_report_gui, fg="black", font=("Times New Roman", 13), bd=4)
report_button.pack(side=BOTTOM, padx=10, pady=10)


#def clear_text():
#    output_txt.delete(1.0, END)
#    root.after(50000, clear_text)  
output_txt = Text(root, width=50, height=8, bg="black", fg="pale green")
output_txt.place(x=1350, y=126)

def auto_scroll_up():
    output_txt.yview_scroll(-1, "units")
    if float(output_txt.yview()[0]) > 0.0:
        root.after(1100, auto_scroll_up)
    else:
        root.after(1100, auto_scroll_down)

def auto_scroll_down():
    output_txt.yview_scroll(1, "units")
    if float(output_txt.yview()[1]) < 1.0:
        root.after(1100, auto_scroll_down)
    else:
        root.after(1100, auto_scroll_up)

# Start with scrolling up
auto_scroll_up()

#root.after(50000, clear_text)  # Schedule the first call to clear_text after 60 seconds
Bitlabel = Label(root, text="[#] Bit locations [#] ", fg= "white", bg="#2C2C2C", font=("Times New Roman", 15))
Bitlabel.place(x= 1462, y= 98)
#############################################################################################################
# Define the BinaryDisplay class
class BinaryDisplay(Frame):
    BITS_COUNT = 32

    def __init__(self, master=None):
        super().__init__(master, bg='black')  # Set the background color to black
        self.master = master
        self.create_widgets()
        self.bits = [0] * self.BITS_COUNT

    def create_widgets(self):
        self.bit_labels = []
        for i in range(self.BITS_COUNT):
            label = Label(self, text="0", width=2, bg="black", fg="white", font=("Times New Roman", 15),
                             relief="solid", borderwidth=1)
            label.grid(row=1, column=i, padx=0, pady=3)
            self.bit_labels.append(label)

    def set_bit(self, index, value):
        if 0 <= index < self.BITS_COUNT and value in (0, 1):
            self.bits[index] = value
            self.bit_labels[index].config(text=str(value))
            if 0 <= index <= 31:
                self.bit_labels[index].config(bg="sea green")

    def clear_bit(self, index):
        self.set_bit(index, 0)

    def set_bits_from_hex_values(self, *hex_values):
        for hex_value in hex_values:
            binary_value = bin(int(hex_value, 16))[2:].zfill(32)
            for i, bit in enumerate(hex_value):
                self.set_bit(i, int(bit))

# Function to update the BinaryDisplay
def update_binary_display(hex_value):
    binary_display.set_bits_from_hex_values(hex_value)

# Create BinaryDisplay instance
binary_display = BinaryDisplay(master=root)
# Place BinaryDisplay at a specific location
binary_display.place(x=1035, y=270)

###########################################################################################################
frame = Frame(root, borderwidth=3, relief="solid", padx=10, pady=10, bg="#8B3A62")
frame.place(x=10, y=744)

cpu_label = Label(frame, text='', font=('Times New Roman', 12))
cpu_label.pack()

memory_label = Label(frame, text='', font=('Times New Roman', 12))
memory_label.pack()

update_data()
#############################################################
    
text_box = ScrolledText(root, height=30, width=100, wrap=WORD, bg="black", fg="white")
text_box.pack(padx=15, pady=10, side=RIGHT, anchor=SE)
#################################################################################

with open("PCIe Register Data.json", "r") as file:
    jsonData = json.load(file)

def print_Help():
    output_text.delete(1.0, END)
    # Access jsonData here
    keys = ["Linux_Help", "guide"]
    info = jsonData[keys[0]][keys[1]]
    output_text.insert("end", info[0] + "\n")  # Print the first line separately
    for line in info[1:]:  # Iterate through the remaining lines
        output_text.insert("end", line + "\n")
    output_text.insert("end", "\n")
########################################################################################    
Ubuntu_photo = Image.open("Linux_Minotaur.jpg")
Resized_image = Ubuntu_photo.resize((140, 155), Image.ANTIALIAS)
converted_image = ImageTk.PhotoImage(Resized_image)


Button_linux = Button(root, image=converted_image, width=140, height=140, compound="top", borderwidth=5, command=print_Help)
Button_linux.place(x=20, y=525)

overlay_frame = Label(root, text="Linux Help",fg ="black", font=("Times New Roman", 14))
overlay_frame.place(x=55, y=500)

#####################################################################################
frame = LabelFrame(root, text="   Linux PCIe Commands", padx=20, pady=5, bg="#545454", fg="white")
frame.pack(padx=10, pady=5, side=LEFT, anchor=NW)

###############################################
list_box = Listbox(root, selectmode=SINGLE, height=11, width=95)
list_box.pack(padx=10, pady=10, side=TOP, anchor=NW)

######################################################################################################
# Create and place the text box (output_text)
output_text = ScrolledText(root, width=93, height=30, bg="black", fg="white")
output_text.pack(padx=10, pady=15, side=LEFT, anchor=SW) 
################################################## main buttons
button_width = 12
button_colour = "#8B3A62"

button1 = Button(frame, text="List PCI Devices", command=llist, width=button_width, bg=button_colour, relief="raised", bd=4)
button1.pack(anchor=N, padx=5, pady=0)
button0 = Button(frame, text="lspci -s", command=lspci_s, width=button_width, bg=button_colour)
button0.pack(anchor=N, padx=5, pady=0)
button3 = Button(frame, text="lspci -x", command=lspci_x, width=button_width, bg=button_colour)
button3.pack(anchor=N, padx=5, pady=0)
button4 = Button(frame, text="lspci -k", command=lspci_k, width=button_width, bg=button_colour)
button4.pack(anchor=N, padx=5, pady=0)
button5 = Button(frame, text="lspci -v", command=lspci_v, width=button_width, bg=button_colour)
button5.pack(anchor=N, padx=5, pady=0)
button6 = Button(frame, text="lspci -n", command=lspci_n, width=button_width, bg=button_colour)
button6.pack(anchor=N, padx=5, pady=0)
button11 = Button(frame, text="lspci -vvv", command=lspci_vvv, width=button_width, bg=button_colour)
button11.pack(anchor=N, padx=5, pady=0)
button8 = Button(frame, text="lspci -vmm", command=lspci_vmm, width=button_width, bg=button_colour)
button8.pack(anchor=N, padx=5, pady=0)
button9 = Button(frame, text="clear", command=clear_output, width=button_width, bg=button_colour)
button9.pack(anchor=N, padx=5, pady=0)


# Run the GUI main loop
root.mainloop()